In [1]:
from google.colab import files
import pandas as pd

In [2]:
uploaded = files.upload()
uploaded = files.upload()

Saving books.csv to books.csv


Saving ratings.csv to ratings.csv


In [3]:
books = pd.read_csv("books.csv")
ratings = pd.read_csv("ratings.csv")
print("Books sample:")
print(books.head())
print("\nRatings sample:")
print(ratings.head())

Books sample:
   book_id   title     author     genre
0        1  Book 1  Author 15   Mystery
1        2  Book 2   Author 2   Mystery
2        3  Book 3   Author 3  Thriller
3        4  Book 4  Author 10   Mystery
4        5  Book 5  Author 19   Romance

Ratings sample:
   user_id  book_id  rating
0        6       65       5
1        5       15       3
2       16       77       4
3        8       29       3
4        8       95       5


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

books['features'] = books['title'] + " " + books['author'] + " " + books['genre']

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(books['features'])

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

def recommend_books(title, cosine_sim=cosine_sim, n=5):
    if title not in books['title'].values:
        return ["❌ Book not found in dataset."]
    idx = books[books['title'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:n+1]
    book_indices = [i[0] for i in sim_scores]
    return books['title'].iloc[book_indices].tolist()

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

user_item_matrix = ratings.pivot_table(index="user_id", columns="book_id", values="rating").fillna(0)
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

def recommend_for_user(user_id, n=5):
    if user_id not in user_similarity_df.index:
        return ["❌ User not found in dataset."]

    sim_scores = user_similarity_df[user_id].sort_values(ascending=False)
    similar_users = sim_scores.index[1:6]

    similar_users_ratings = ratings[ratings["user_id"].isin(similar_users)]
    user_books = ratings[ratings["user_id"] == user_id]["book_id"].tolist()
    candidate_books = similar_users_ratings[~similar_users_ratings["book_id"].isin(user_books)]

    recommendations = candidate_books.groupby("book_id")["rating"].mean().sort_values(ascending=False).head(n)
    return books[books["book_id"].isin(recommendations.index)][["title","author","genre"]].to_dict(orient="records")

In [6]:
def recommend_by_genre(genre, n=5):
    genre_books = books[books["genre"].str.lower() == genre.lower()]
    if genre_books.empty:
        return ["❌ Genre not found."]
    # Average rating per book
    avg_ratings = ratings.groupby("book_id")["rating"].mean()
    genre_books = genre_books.merge(avg_ratings, left_on="book_id", right_index=True, how="left")
    top_books = genre_books.sort_values("rating", ascending=False).head(n)
    return top_books[["title","author","genre","rating"]].to_dict(orient="records")

In [7]:
def best_rated_books(n=5, m=10):
    avg_ratings = ratings.groupby("book_id")["rating"].mean()
    count_ratings = ratings.groupby("book_id")["rating"].count()

    # Global average rating
    C = avg_ratings.mean()

    # DataFrame of ratings
    rating_summary = pd.DataFrame({
        "book_id": avg_ratings.index,
        "R": avg_ratings.values,
        "v": count_ratings.values
    })

    # Apply weighted rating
    rating_summary["score"] = (rating_summary["v"] / (rating_summary["v"] + m)) * rating_summary["R"] + \
                              (m / (rating_summary["v"] + m)) * C

    # Sort by score
    top_books = books.merge(rating_summary, on="book_id").sort_values("score", ascending=False).head(n)
    return top_books[["title","author","genre","R","v","score"]].to_dict(orient="records")


In [8]:
while True:
    print("\n--- Book Recommendation System ---")
    choice = input("Choose option: \n1. Similar books \n2. User-based recommendations \n3. Genre-based recommendations \n4. Best-rated books \n5. Exit\nEnter choice: ")

    if choice == "1":
        book_title = input("Enter a book title: ")
        recs = recommend_books(book_title)
        print(f"\n📚 Books similar to '{book_title}':")
        for r in recs:
            print(" -", r)

    elif choice == "2":
        try:
            user_id = int(input("Enter a user ID: "))
            recs = recommend_for_user(user_id)
            print(f"\n⭐ Recommendations for User {user_id}:")
            for r in recs:
                print(f" - {r['title']} ({r['author']}, {r['genre']})")
        except:
            print("❌ Invalid User ID")

    elif choice == "3":
        genre = input("Enter a genre (e.g., Fantasy, Romance, Thriller): ")
        recs = recommend_by_genre(genre)
        print(f"\n🎭 Top {len(recs)} books in genre '{genre}':")
        for r in recs:
            print(f" - {r['title']} ({r['author']}, {r['genre']}) ⭐ {r['rating']:.2f}")

    elif choice == "4":
        recs = best_rated_books()
        print("\n🏆 Best-rated books overall:")
        for r in recs:
            for r in recs:
              print(f" - {r['title']} ({r['author']}, {r['genre']}) ⭐ {r['R']:.2f} | {r['v']} ratings | Score: {r['score']:.2f}")


    elif choice == "5":
        print("Goodbye 👋")
        break
    else:
        print("❌ Invalid choice, try again.")


--- Book Recommendation System ---
Choose option: 
1. Similar books 
2. User-based recommendations 
3. Genre-based recommendations 
4. Best-rated books 
5. Exit
Enter choice: 1
Enter a book title: Book 5

📚 Books similar to 'Book 5':
 - Book 13
 - Book 19
 - Book 11
 - Book 93
 - Book 99

--- Book Recommendation System ---
Choose option: 
1. Similar books 
2. User-based recommendations 
3. Genre-based recommendations 
4. Best-rated books 
5. Exit
Enter choice: 2
Enter a user ID: 4

⭐ Recommendations for User 4:
 - Book 38 (Author 6, Romance)
 - Book 45 (Author 17, Mystery)
 - Book 86 (Author 18, Non-Fiction)
 - Book 87 (Author 17, Mystery)
 - Book 100 (Author 17, Thriller)

--- Book Recommendation System ---
Choose option: 
1. Similar books 
2. User-based recommendations 
3. Genre-based recommendations 
4. Best-rated books 
5. Exit
Enter choice: 3
Enter a genre (e.g., Fantasy, Romance, Thriller): Fantasy

🎭 Top 5 books in genre 'Fantasy':
 - Book 27 (Author 7, Fantasy) ⭐ 5.00
 - Book 